In [ ]:
!mkdir ./data/
!wget -O ./data/hard-hat-workers-dataset.zip "https://app.roboflow.com/ds/QTYQxhu7es?key=*ENTER KEY*"
!unzip ./data/hard-hat-workers-dataset.zip -d ./data/
!ls ./data/

In [ ]:
# %cd /kaggle/working/
# !rm -rf dab-detr
# !git clone https://github.com/NinthAutumn/dab-detr.git
# %cd dab-detr
# !pip install -r requirements.txt
# %cd models/dab_deformable_detr/ops
# !python setup.py build install
# # unit test (should see all checking is True)
# # !python test.py
# %cd ../../../
# !mkdir ../data/annotations
# !mv ../data/train/_annotations.coco.json ../data/annotations/instances_train2017.json
# !mv ../data/valid/_annotations.coco.json ../data/annotations/instances_val2017.json
# !mv ../data/train ../data/train2017
# !mv ../data/valid ../data/val2017
# import wandb
# !wandb login 38d3d0b9ff8932c38ed3c492b184f347f107a30a

In [ ]:
# !python main.py -m dab_detr --output_dir ../save --batch_size 5 --coco_path ../data --epoch 15 --save_log --save_results --lr_drop 40  # replace the args to your checkpoint path

In [ ]:

# wandb.init()

In [ ]:

# !python main_dino.py --help


In [ ]:
# !git clone https://github.com/Visual-Behavior/detr-tensorflow.git    

import sys
# # Set the path to detr_tf
sys.path.append("../input/detr-tensorflow/detr-tensorflow-main")
# import detr_tf
import wandb
import matplotlib
import numpy
!pip install pycocotools
import pycocotools
import imageio
import pandas
import tensorflow as tf
from detr_tf import training


In [ ]:
# !pip install torch==1.8.0 torchvision==0.9.0 -qq

# import torch, torchvision
# print(torch.__version__, torch.cuda.is_available())

In [ ]:
# pretrained = True

# if pretrained:
#     # Get pretrained weights
#     checkpoint = torch.hub.load_state_dict_from_url(
#                 url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
#                 map_location='cpu',
#                 check_hash=True)

#     # Remove class weights
#     del checkpoint["model"]["class_embed.weight"]
#     del checkpoint["model"]["class_embed.bias"]

#     # SaveOGH
#     torch.save(checkpoint,
#                'detr-r50_no-class-head.pth')

In [ ]:
# dataset_file = "coco" # alternatively, implement your own coco-type dataset loader in datasets and add this "key" to datasets/__init__.py

# dataDir='./data/' # should lead to a directory with a train2017 and val2017 folder as well as an annotations folder
# num_classes = 3 # this int should be the actual number of classes + 1 (for no class)

# outDir = 'outputs'
# resume = "detr-r50_no-class-head.pth" if pretrained else ""


In [ ]:
from detr_tf.training_config import TrainingConfig, DataConfig
from detr_tf.data import load_tfcsv_dataset

from os.path import expanduser
import os
class TrainConfig(TrainingConfig):
    def __init__(self):
        super().__init__()
        self.data_dir = "./data/"
        self.data = DataConfig(data_dir=os.path.abspath( "./data/"), img_dir="train", ann_file="train/_annotations.csv")
        self.batch_size = 1
        self.image_size = (416, 416)
        self.target_batch = 8
        self.log=True

class ValidConfig(TrainingConfig):
    def __init__(self):
        super().__init__()
        self.data_dir = "./data/"
        self.data = DataConfig(data_dir=os.path.abspath( "./data/"), img_dir="test", ann_file="test/_annotations.csv")
        self.batch_size = 1
        self.image_size = (416, 416)
        self.log =True
        self.target_batch = 8

train_config = TrainConfig()
valid_config = ValidConfig()
train_iterator, class_names = load_tfcsv_dataset(train_config, train_config.batch_size, augmentation=True,exclude=["person"])
valid_iterator, class_names = load_tfcsv_dataset(valid_config, valid_config.batch_size, augmentation=True,exclude=["person"])
print(class_names)

In [ ]:
from detr_tf.networks.detr import get_detr_model
# Load the pretrained DETR model with new heads at the top
# include_top: We do not include the last layers that predicts the bbox pos and class (include_top=False)
# weights: Use the "detr" weight to init the model
detr = get_detr_model(train_config, include_top=False, nb_class=3, weights="detr")
detr.summary()

In [ ]:
train_config.train_backbone = tf.Variable(False)
train_config.train_nlayers = tf.Variable(True)
train_config.train_transformers = tf.Variable(True)
train_config.transformers_lr.assign(1e-6)
train_config.nlayers_lr.assign(1e-5)
from detr_tf.optimizers import setup_optimizers
# Setup the optimziers and the trainable variables
optimzers = setup_optimizers(detr, train_config)
!wandb login 38d3d0b9ff8932c38ed3c492b184f347f107a30a
wandb.init()

In [ ]:
# from detr_tf.inference import get_model_inference, numpy_bbox_to_image
# import matplotlib.pyplot as plt
# import numpy as np


# for valid_images, target_bbox, target_class in valid_iterator:
    
#     m_outputs = detr(valid_images, training=False)
#     predicted_bbox, predicted_labels, predicted_scores = get_model_inference(m_outputs, config.background_class, bbox_format="xy_center")

#     result = numpy_bbox_to_image(
#         np.array(valid_images[0]),
#         np.array(predicted_bbox),
#         np.array(predicted_labels),
#         scores=np.array(predicted_scores),
#         class_name=class_names, 
#         config=config
#     )
#     plt.imshow(result)
#     plt.show()
# #     break



In [ ]:
for epoch in range(0, 10):
    training.fit(detr, train_iterator, optimzers, train_config, epoch_nb=epoch, class_names=class_names)

In [ ]:
detr.save_weights('model.h5')

In [ ]:
class ValidConfig(TrainingConfig):
    def __init__(self):
        super().__init__()
        self.data_dir = "./data/"
        self.data = DataConfig(data_dir=os.path.abspath( "./data/"), img_dir="validation", ann_file="validation/_annotations.csv")
        self.batch_size = 1
        self.image_size = (416, 416)
        self.log =True
        self.target_batch = 8

valid_config = ValidConfig()

In [ ]:
training.eval(detr,valid_iterator,valid_config ,class_name=class_names,evaluation_step=200)